In [53]:
# загрузка нужных библиотек
!pip install transliterate
!pip install geopandas
!pip install pandas
!pip install datetime
!pip install shapely
!pip installnumpy

     ---------------------------------------- 1.1/1.1 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 6.1/6.1 MB 11.1 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 10.1 MB/s eta 0:00:00
     --------------------------------------- 22.9/22.9 MB 10.5 MB/s eta 0:00:00


In [54]:
# импорт библиотек
import pandas as pd
from datetime import datetime
from transliterate import translit
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

In [27]:
# импорт листов экселя и преобразование в датафрейм
exc = pd.ExcelFile('hotels_parsing_result.xlsx')
plat_1 = exc.parse('platform1', parse_dates=['create_time'], index_col='id')
display(plat_1.head())
plat_2 = exc.parse('platfrom2', parse_dates=['create_time'], index_col='id')
display(plat_2.head())

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
id,,,,,,,,,,,,,,,,,,,,
81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526
86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5,2,NaN,8.0,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150
86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10,2,4.0,15.0,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445
86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2,2,NaN,15.0,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596
85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0,2,2.0,23.0,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686


,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
id,,,,,,,,,,,,,,,,,,,,
4608,2023-09-14 15:46:07.855977,"Уют, гостиница",hotel,NaN,"Россия, Республика Башкортостан, Аскинский рай...",NaN,NaN,NaN,2,NaN,5.0,NaN,NaN,NaN,NaN,NaN,70000001064499291,2023-09-14 22:44:57.013238,NaN,NaN
4815,2023-09-14 15:46:48.332492,Гостиница Ашкадар,NaN,sterlitamak,"Россия, Республика Башкортостан, Стерлитамак г...",NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7600610350202940,2023-09-14 22:44:57.054250,NaN,NaN
4750,2023-09-14 15:46:35.659931,"Островок, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Гафурийский р...",NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000001076845858,2023-09-14 22:44:57.043234,NaN,NaN
4786,2023-09-14 15:46:42.672665,"Ардан, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Баймакский ра...",NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000001075946655,2023-09-14 22:44:57.049244,NaN,NaN
16310,2023-09-15 11:56:48.785266,Гостиница,hotel,beloretsk,"Россия, Республика Башкортостан, Белорецкий ра...",NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70000001031030911,2023-09-15 18:55:23.191778,53.969912,58.400557


In [28]:
# объединение двух датафреймов в один и создание нового столбца platform, чтобы сохранился источник
plat_1['platform'] = 1
plat_2['platform'] = 2
hotels = pd.concat([plat_1, plat_2], axis = 0)
hotels

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,contact_social,description,email,phone,website,uid,parsing_time,lat,lon,platform
id,,,,,,,,,,,,,,,,,,,,,
81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526,1
86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150,1
86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10.0,2,4.0,...,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445,1
86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596,1
85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0.0,2,2.0,...,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49848,2023-09-19 21:46:11.061452,"Караидель, физкультурно-оздоровительный комплекс",hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,NaN,...,"[{""social"": ""vk"", ""contact"": ""https://vk.com/c...",NaN,NaN,+73472827979,www.kara-idel.ru,70000001069863555,2023-09-20 04:43:50.769707,55.794670,56.864993,2
49851,2023-09-19 21:46:11.764997,"Lounge villa, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Туймазинский ...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069856457,2023-09-20 04:43:51.527665,54.434405,54.018789,2
49852,2023-09-19 21:46:11.981114,"Лена, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Илишевский ра...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069870275,2023-09-20 04:43:51.802655,55.288257,54.483841,2


In [29]:
#сохранение названий столбцов в список 
column_names = hotels.columns
print(column_names)

Index(['create_time', 'title', 'hotel_type_original', 'city', 'address',
       'rating', 'rating_5', 'review_count', 'region', 'star_rating',
       'rooms_count', 'contact_social', 'description', 'email', 'phone',
       'website', 'uid', 'parsing_time', 'lat', 'lon', 'platform'],
      dtype='object')


In [30]:
#цикл который проходит по названию столбцов и заполняет нулевые ячейки значениями сверху
for i in column_names:
    hotels[i] = hotels.groupby('uid')[i].transform(lambda x: x.bfill())
hotels

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,contact_social,description,email,phone,website,uid,parsing_time,lat,lon,platform
id,,,,,,,,,,,,,,,,,,,,,
81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526,1
86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150,1
86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10.0,2,4.0,...,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445,1
86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596,1
85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0.0,2,2.0,...,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49848,2023-09-19 21:46:11.061452,"Караидель, физкультурно-оздоровительный комплекс",hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,NaN,...,"[{""social"": ""vk"", ""contact"": ""https://vk.com/c...",NaN,NaN,+73472827979,www.kara-idel.ru,70000001069863555,2023-09-20 04:43:50.769707,55.794670,56.864993,2
49851,2023-09-19 21:46:11.764997,"Lounge villa, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Туймазинский ...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069856457,2023-09-20 04:43:51.527665,54.434405,54.018789,2
49852,2023-09-19 21:46:11.981114,"Лена, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Илишевский ра...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069870275,2023-09-20 04:43:51.802655,55.288257,54.483841,2


In [31]:
#удаление строк со старыми записями
filtered = hotels.groupby('uid')['create_time'].transform('max')
hotels = hotels[hotels['create_time'] == filtered]
hotels

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,contact_social,description,email,phone,website,uid,parsing_time,lat,lon,platform
id,,,,,,,,,,,,,,,,,,,,,
81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526,1
86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150,1
86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10.0,2,4.0,...,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445,1
86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596,1
85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0.0,2,2.0,...,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67995,2023-09-29 15:27:50.983083,Мед от пчел,hotel_rest,NaN,"Россия, Республика Башкортостан, Мишкинский ра...",5.0,5.00,1.0,2,NaN,...,"[{""social"": ""vk"", ""contact"": ""https://vk.com/m...",NaN,NaN,+79872414963,Забронировать,70000001044671152,2023-09-29 22:25:26.647427,NaN,NaN,2
67996,2023-09-29 15:27:51.234779,"Lounge villa, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Туймазинский ...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069856457,2023-09-29 22:25:26.905886,54.434405,54.018789,2
67997,2023-09-29 15:27:51.455089,"Лена, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Илишевский ра...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069870275,2023-09-29 22:25:27.155569,55.288257,54.483841,2


In [32]:
#исправление типа отеля, чтобы был одинаковый тип
hotels.loc[:, 'hotel_type_original'] = hotels['hotel_type_original'].str.capitalize()
hotels

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,contact_social,description,email,phone,website,uid,parsing_time,lat,lon,platform
id,,,,,,,,,,,,,,,,,,,,,
81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526,1
86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150,1
86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10.0,2,4.0,...,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445,1
86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596,1
85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0.0,2,2.0,...,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67995,2023-09-29 15:27:50.983083,Мед от пчел,Hotel_rest,NaN,"Россия, Республика Башкортостан, Мишкинский ра...",5.0,5.00,1.0,2,NaN,...,"[{""social"": ""vk"", ""contact"": ""https://vk.com/m...",NaN,NaN,+79872414963,Забронировать,70000001044671152,2023-09-29 22:25:26.647427,NaN,NaN,2
67996,2023-09-29 15:27:51.234779,"Lounge villa, база отдыха",Hotel_rest,NaN,"Россия, Республика Башкортостан, Туймазинский ...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069856457,2023-09-29 22:25:26.905886,54.434405,54.018789,2
67997,2023-09-29 15:27:51.455089,"Лена, база отдыха",Hotel_rest,NaN,"Россия, Республика Башкортостан, Илишевский ра...",NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,+79236157271,Забронировать,70000001069870275,2023-09-29 22:25:27.155569,55.288257,54.483841,2


In [34]:
#исправление названий города автоматическим переводом с латиницы на кириллицу
uniq_cities = hotels['city'][hotels['platform'] == 2].dropna().unique()
city_translation = {city: translit(city, 'ru').capitalize() for city in uniq_cities}
print(city_translation)
# ручное подправление
city_translation['oktyabrskiy'] = 'Октябрьский'
hotels.loc[:, 'city'] = hotels['city'].replace(city_translation)

{'Уфа': 'Уфа', 'Магнитогорск': 'Магнитогорск', 'Стерлитамак': 'Стерлитамак', 'Октябрьский': 'Октябрьский', 'Нефтекамск': 'Нефтекамск', 'Салават': 'Салават', 'Кумертау': 'Кумертау', 'Белорецк': 'Белорецк', 'Ижевск': 'Ижевск'}


In [35]:
# редактирование адреса, чтобы он был под один шаблон
def first_plat_add(adress):
    spl_adress = adress.split(", ")
    final_adress = spl_adress[0] + ', ' + spl_adress[1]
    return final_adress
def second_plat_add(adress):
    spl_adress = adress.split(", ")
    final_adress = spl_adress[-2] + ', ' + spl_adress[-1]
    return final_adress
hotels.loc[:, 'address'] = hotels.apply(lambda row: first_plat_add(row['address']) if row['platform'] == 1 else second_plat_add(row['address']), axis=1)
hotels['address']

id
81842                         улица Приморская, д. 31
86341                           улица Мушникова, д.28
86420                     Коммунистическая ул., д. 53
86421                            Учалинская улица, 37
85898                            улица Дружбы, д.28 Д
                             ...                     
67995                 Мишкинский район, д. Курманаево
67996     Республика Башкортостан, Туймазинский район
67997                              Луговая улица, ст1
67998    Республика Башкортостан, Краснокамский район
67999                     Стерлитамак, Западный район
Name: address, Length: 6071, dtype: object

In [45]:
# создание столбца со списком из контактов отеля
hotels.loc[:, 'email'] = hotels['email'].fillna('')
hotels.loc[:, 'phone'] = hotels['phone'].fillna('')
hotels.loc[:, 'website'] = hotels['website'].fillna('')
hotels.loc[:, 'contacts'] = (hotels['email'] + hotels['phone'] + hotels['website']).tolist()

In [71]:
# топ 10 отелей к каждой платформе по количеству номеров
hotels.loc[:, 'phone_count'] = hotels['phone'].apply(lambda x: len(str(x).split(', ')))
top_10_phones = hotels.groupby('platform').apply(lambda group: group.nlargest(10, 'phone_count'))
top_10_phones

create_time                             title  \
platform id                                                                   
1        81842 2023-09-26 14:45:42.910962  Апартаменты Baltia Западный Пляж   
         86341 2023-09-26 15:01:55.184815                     Отель Авиатор   
         86420 2023-09-26 15:02:12.005085                         Клеопатра   
         86421 2023-09-26 15:02:12.225334                           Татьяна   
         85898 2023-09-26 15:00:19.217359                Мини-Отель Чемодан   
         85899 2023-09-26 15:00:19.415692   Hilton Garden Inn Ufa Riverside   
         85900 2023-09-26 15:00:19.631849       Отель Hampton by Hilton Уфа   
         85901 2023-09-26 15:00:19.871941                   Отель ВолнаСити   
         85902 2023-09-26 15:00:20.095657             AZIMUT Сити Отель Уфа   
         85903 2023-09-26 15:00:20.316568                   Отель Ural Taur   
2        67446 2023-09-29 15:25:29.099018                              Эдем   
         67447 2023-09-29 15:25:29.399452          Тихорецкий, гостевой дом   
         67448 2023-09-29 15:25:29.671305             СЕМЬЯ, мини-гостиница   
         67449 2023-09-29 15:25:29.947243                Маракеш, гостиница   
         67450 2023-09-29 15:25:30.231251                     Корона, отель   
         67451 2023-09-29 15:25:30.526616                 Молодость, студия   
         67452 2023-09-29 15:25:30.827197  Ай да вкус, гостиничный комплекс   
         67453 2023-09-29 15:25:31.114951                  Уршак, гостиница   
         67454 2023-09-29 15:25:31.386927                 Огонек, гостиница   
         67455 2023-09-29 15:25:31.646833     Каменный цветок, гостевой дом   

               hotel_type_original          city  \
platform id                                        
1        81842           Apartment  Зеленоградск   
         86341               Hotel           Уфа   
         86420               Hotel           Уфа   
         86421               Hotel         Сибай   
         85898          Mini-hotel   Стерлитамак   
         85899               Hotel           Уфа   
         85900               Hotel           Уфа   
         85901               Hotel           Уфа   
         85902               Hotel           Уфа   
         85903               Hotel    Нефтекамск   
2        67446               Hotel           Уфа   
         67447               Hotel           Уфа   
         67448               Hotel           Уфа   
         67449               Hotel           Уфа   
         67450               Hotel           Уфа   
         67451               Hotel           Уфа   
         67452               Hotel           Уфа   
         67453               Hotel           Уфа   
         67454               Hotel           Уфа   
         67455               Hotel  Магнитогорск   

                                     address  rating  rating_5  review_count  \
platform id                                                                    
1        81842       улица Приморская, д. 31    10.0      5.00           0.0   
         86341         улица Мушникова, д.28     8.7      4.35           5.0   
         86420   Коммунистическая ул., д. 53     6.3      3.15          10.0   
         86421          Учалинская улица, 37    10.0      5.00           2.0   
         85898          улица Дружбы, д.28 Д     8.0      4.00           0.0   
         85899         улица Аксакова 4, Уфа     9.1      4.55         616.0   
         85900      улица 50 лет Октября, 17     9.2      4.60         460.0   
         85901     улица Заки Валиди, д.64/2     8.1      4.05          26.0   
         85902           пр-т Октября, д. 81     8.7      4.35         988.0   
         85903      Юбилейный проспект, д.18     8.5      4.25           0.0   
2        67446                  Кирова, 15/1     NaN       NaN           NaN   
         67447                Тихорецкая, 18     5.0      5.00          11.0   
         67448       Средн

In [50]:
# топ 10 отелей к каждой платформе по количеству отзывов
top_10_review = hotels.groupby('platform').apply(lambda group: group.nlargest(10, 'review_count'))
top_10_review

create_time  \
platform id                                 
1        82091 2023-09-26 14:46:37.319096   
         85902 2023-09-26 15:00:20.095657   
         82175 2023-09-26 14:46:55.755653   
         81762 2023-09-26 14:45:25.258898   
         85899 2023-09-26 15:00:19.415692   
         85905 2023-09-26 15:00:20.752567   
         85900 2023-09-26 15:00:19.631849   
         86117 2023-09-26 15:01:06.720495   
         81759 2023-09-26 14:45:24.594611   
         81926 2023-09-26 14:46:01.087115   
2        67479 2023-09-29 15:25:38.439362   
         67476 2023-09-29 15:25:37.371242   
         67485 2023-09-29 15:25:40.275563   
         67484 2023-09-29 15:25:40.066673   
         67480 2023-09-29 15:25:38.779367   
         67487 2023-09-29 15:25:40.901918   
         67478 2023-09-29 15:25:38.091467   
         67477 2023-09-29 15:25:37.739400   
         67502 2023-09-29 15:25:45.335533   
         67463 2023-09-29 15:25:33.979152   

                                                      title  \
platform id                                                   
1        82091               Radisson Blu Hotel Kaliningrad   
         85902                        AZIMUT Сити Отель Уфа   
         82175                  Отель Кайзерхоф (Kaiserhof)   
         81762                 Отель Ibis Калининград Центр   
         85899              Hilton Garden Inn Ufa Riverside   
         85905                       Отель АМАКС Сити-отель   
         85900                  Отель Hampton by Hilton Уфа   
         86117         Гостиничный Комплекс Президент Отель   
         81759           Гостиница Мартон Палас Калининград   
         81926                       Гостиница "Шкиперская"   
2        67479       Hilton Garden Inn Ufa Riverside, отель   
         67476               Башкирия, гостиничный комплекс   
         67485                 Hampton by hilton Ufa, отель   
         67484               Урал тау, гостиничный комплекс   
         67480                        AZIMUT Сити Отель Уфа   
         67487                  Амакс Сити-отель, гостиница   
         67478        Президент отель, гостиничный комплекс   
         67477  Sheratonplaza Ufa Congress Hotel, гостиница   
         67502                             ВолнаСити, отель   
         67463                  Свояк, гостиничный комплекс   

               hotel_type_original         city                       address  \
platform id                                                                     
1        82091               Hotel  Калининград         площадь Победы, д. 10   
         85902               Hotel          Уфа           пр-т Октября, д. 81   
         82175               Hotel  Калининград      улица Октябрьская, д.6 А   
         81762               Hotel  Калининград     Московский проспект, д.52   
         85899               Hotel          Уфа         улица Аксакова 4, Уфа   
         85905               Hotel          Уфа    улица Рихарда Зорге, д. 17   
         85900               Hotel          Уфа      улица 50 лет Октября, 17   
         86117               Hotel          Уфа             улица Авроры, д.2   
         81759               Hotel  Калининград  Большевистский переулок, д.3   
         81926               Hotel  Калининград           ул. Октябрьская, 4а   
2        67479               Hotel          Уфа                   Аксакова, 4   
         67476               Hotel          Уфа                 Ленина, 25/29   
         67485               Hotel          Уфа          50-летия Октября, 17   
         67484               Hotel          Уфа                    Гоголя, 72   
         67480               Hotel          Уфа          Октября проспект, 81   
         67487               Hotel          Уфа             Рихарда Зорге, 17   
         67478               Hotel          Уфа                     Авроры, 2   
         67477               Hotel          Уфа                     Цюрупы, 7   
         67502               Hot

In [63]:
# создание координат, сетки и подсчёт количества отелей в каждом квадрате 1 на 1 км
lats = hotels['lat'].tolist()
lons = hotels['lon'].tolist()
geometry = [Point(lon, lat) for lat, lon in zip(lats, lons)]
gdf = gpd.GeoDataFrame(hotels, geometry=geometry)
grid_size = 1 / 111.32
x_range = np.arange(gdf['geometry'].bounds['minx'].min(), gdf['geometry'].bounds['maxx'].max(), grid_size)
y_range = np.arange(gdf['geometry'].bounds['miny'].min(), gdf['geometry'].bounds['maxy'].max(), grid_size)
grid = gpd.GeoDataFrame(geometry=[Point(x, y) for x in x_range for y in y_range], columns=['geometry'])
result = gpd.sjoin(gdf, grid, how='right', op='intersects')
hotel_counts = result.groupby('geometry').size().reset_index(name='hotel_count')
max_hotel_square = hotel_counts.loc[hotel_counts['hotel_count'].idxmax()]
max_hotel_square

D:\python\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


geometry       POINT (19.876875 51.858433)
hotel_count                              1
Name: 0, dtype: object

In [64]:
# создание множеств с индексами отелей
uid_1 = hotels[hotels['platform'] == 1]['uid'].unique()
uid_2 = hotels[hotels['platform'] == 2]['uid'].unique()
set_1 = set(uid_1)
set_2 = set(uid_2)

In [68]:
# отображение отелей которые есть в первой платформе, но нет во второй
in_1_not_in_2 = hotels[~hotels['uid'].isin(set_2)]
in_1_not_in_2

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,email,phone,website,uid,parsing_time,lat,lon,platform,contacts,phone_count
id,,,,,,,,,,,,,,,,,,,,,
81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31",10.0,5.00,0.0,2,NaN,...,,,,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526,1,,1
86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28",8.7,4.35,5.0,2,NaN,...,,,,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150,1,,1
86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53",6.3,3.15,10.0,2,4.0,...,,,,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445,1,,1
86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37",10.0,5.00,2.0,2,NaN,...,,,,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596,1,,1
85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д",8.0,4.00,0.0,2,2.0,...,,,,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686,1,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86744,2023-09-26 15:03:21.089955,Podushka Hostel,Hostel,Уфа,"Ulitsa Lenina, 9/11",7.5,3.75,0.0,2,NaN,...,,,,8661195,2023-09-26 20:56:02.013106,54.727528,55.947640,1,,1
86790,2023-09-26 15:03:30.721937,Lux apartament UFA,Apartment,Уфа,"Ulitsa Mingazheva 102, 450078 Ufa",0.0,0.00,0.0,2,NaN,...,,,,6297197,2023-09-26 20:56:02.013106,54.727356,55.968070,1,,1
87120,2023-09-26 15:04:39.726907,Биатлон,Hotel,Уфа,"ул. Комарова 1, Уфа",0.0,0.00,0.0,2,NaN,...,,,,8753471,2023-09-26 20:56:02.013106,54.805300,56.036000,1,,1


In [69]:
# отображение отелей которые есть и в первой и во второй платформах
in_1_in_2 = hotels[hotels['uid'].isin(set_2 & set_1)]
in_1_in_2

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,email,phone,website,uid,parsing_time,lat,lon,platform,contacts,phone_count
id,,,,,,,,,,,,,,,,,,,,,


In [70]:
# топ 10 отелей по количеству отзывов
top_10_review_all = hotels.nlargest(10, 'review_count')
top_10_review_all

,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,star_rating,...,email,phone,website,uid,parsing_time,lat,lon,platform,contacts,phone_count
id,,,,,,,,,,,,,,,,,,,,,
82091,2023-09-26 14:46:37.319096,Radisson Blu Hotel Kaliningrad,Hotel,Калининград,"площадь Победы, д. 10",9.0,4.50,1201.0,2,4.0,...,,,,7386924,2023-09-26 20:56:02.013106,54.719917,20.504538,1,,1
85902,2023-09-26 15:00:20.095657,AZIMUT Сити Отель Уфа,Hotel,Уфа,"пр-т Октября, д. 81",8.7,4.35,988.0,2,4.0,...,,,,7403650,2023-09-26 20:56:02.013106,54.774017,56.025784,1,,1
82175,2023-09-26 14:46:55.755653,Отель Кайзерхоф (Kaiserhof),Hotel,Калининград,"улица Октябрьская, д.6 А",9.2,4.60,930.0,2,4.0,...,,,,7467265,2023-09-26 20:56:02.013106,54.702940,20.515600,1,,1
81762,2023-09-26 14:45:25.258898,Отель Ibis Калининград Центр,Hotel,Калининград,"Московский проспект, д.52",8.7,4.35,837.0,2,3.0,...,,,,7999515,2023-09-26 20:56:02.013106,54.709034,20.506773,1,,1
67479,2023-09-29 15:25:38.439362,"Hilton Garden Inn Ufa Riverside, отель",Hotel,Уфа,"Аксакова, 4",4.5,4.50,823.0,2,4.0,...,,,www.hilton.ru,70000001017323535,2023-09-29 22:23:03.528795,54.719262,55.930077,2,www.hilton.ru,1
67476,2023-09-29 15:25:37.371242,"Башкирия, гостиничный комплекс",Hotel,Уфа,"Ленина, 25/29",4.8,4.80,780.0,2,4.0,...,,,www.gkbashkortostan.ru,2393065583019903,2023-09-29 22:23:02.714399,54.730703,55.949172,2,www.gkbashkortostan.ru,1
85899,2023-09-26 15:00:19.415692,Hilton Garden Inn Ufa Riverside,Hotel,Уфа,"улица Аксакова 4, Уфа",9.1,4.55,616.0,2,4.0,...,,,,6342481,2023-09-26 20:56:02.013106,54.719320,55.931340,1,,1
85905,2023-09-26 15:00:20.752567,Отель АМАКС Сити-отель,Hotel,Уфа,"улица Рихарда Зорге, д. 17",7.8,3.90,540.0,2,3.0,...,,,,7820894,2023-09-26 20:56:02.013106,54.752770,55.986590,1,,1
85900,2023-09-26 15:00:19.631849,Отель Hampton by Hilton Уфа,Hotel,Уфа,"улица 50 лет Октября, 17",9.2,4.60,460.0,2,3.0,...,,,,6790074,2023-09-26 20:56:02.013106,54.737415,55.969723,1,,1
